# Write SPECFEM3D CMTSOLUTION and STATION files for each EVENT
1. Unpickle events and staions
1. Test taper, pad, and trim 

#### 1. Unpickle events and stations 

In [ ]:
import pickle
from gnam.events.gevents import gevents as gevents
from gnam.events.gstations import gstations as gstations
from gnam.events.munge.knmi import correct_station_depths as csd_f

data_root_dir = '../../../data_notebooks'

#Unpickle events
print('Unpickling Events')
f = open(data_root_dir + '/pickled/events.pickle', 'rb')
dill_events = pickle.load(f)
f.close()
    

#Unpickle stations
print('Unpickling Station Traces')
f = open(data_root_dir + '/pickled/straces.pickle', 'rb')
dill_straces = pickle.load(f)
f.close()

# This is kind of hokey, but it works for now.
# Some of the stations depths do not follow the 
# 50, 100, 150, 200 meter depths -- possibly because
# the boreholes are slanted. To correct for this,
# a hard coded "patch/update" is applied. See the
# code for details and update values.
dill_straces.correct_stations(csd_f)

print('Done')

#### 2. Read and map moment tensors to events

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import obspy
from obspy.core import read
from obspy.signal.trigger import ar_pick
from obspy import Stream
from obspy import Trace
from obspy import UTCDateTime

ekey = 3
ce_st = dill_straces.getStream1(ekey,3)
czne_st = ce_st.copy()
tr = czne_st[0]
t0 = tr.stats.starttime
tN = t0 + 2*16.384
nt = tr.count()
dt = tr.stats.delta
data = tr.data
print('Trace nt=',nt)
print('Trace dt=',dt)
print('Trace shape:\n',data.shape)
tr.taper(max_percentage=1.0,max_length=1,type='slepian',width=0.25,side='right')
tr.trim(t0,tN,pad=True,fill_value=tr.data[-1])
print('Trace last=',tr.data[-1])
nt = tr.count()
dt = tr.stats.delta
data = tr.data
print('Trace nt=',nt)
print('Trace dt=',dt)
print('Trace shape:\n',data.shape)
'''
zdata = np.zeros_like(data)
print('zdata shape:\n',zdata.shape)
temp = np.concatenate((data,zdata),axis=None)
print('Temp shape:\n',temp.shape)
tr.data = temp
nt = tr.count()
dt = tr.stats.delta
data = tr.data
print('Trace nt=',nt)
print('Trace dt=',dt)
print('Trace shape:\n',data.shape)
'''

fig, ax = plt.subplots(1,figsize=(15,4))
ax.plot(ce_st[0].times(), ce_st[0].data,c='black')
ax.plot(tr.times(), tr.data,c='orangered')

plt.grid(b=True, which='major', color='#666666', linestyle='-')
plt.minorticks_on()
plt.grid(b=True, which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

## Finished